In [1]:
from module import TransformerDecoder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [3]:
from char_dataset import CharDataset
shakespeare_dataset = CharDataset("shakespeare.txt", block_size=128)

In [3]:
from char_dataset import CharDataset
shakespeare_dataset = CharDataset("demo.txt", block_size=128)

In [4]:
from torch.utils.data import DataLoader
dataloader = DataLoader(shakespeare_dataset, batch_size=32, shuffle=True)

# Training

In [5]:
decoder = TransformerDecoder(vocab_size=shakespeare_dataset.get_vocab_size(),
                           d_model=128,
                           n_layers=12,
                           n_heads=8,
                           d_ff=100,
                           max_len=512,
                           dropout=0.1).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(decoder.parameters(), lr=0.001)

In [6]:
import torch.nn.functional as F

for epoch in range(1):
    for input_data, target_data in dataloader:
        output = decoder(input_data.to(device))
        output = F.softmax(output, dim=2)
        loss = criterion(output.view(-1, output.size(2)).to(device), target_data.view(-1).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("loss:", loss)

loss: tensor(3.7139, device='cuda:0', grad_fn=<NllLossBackward0>)


In [10]:
decoder.eval()
with torch.no_grad():
    context0 = "O God, Oh my God"*8
    context1 = """We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the"""
    context = context0
    tokenized_context = shakespeare_dataset.to_integer(context).to(device)
    y = decoder.generate(tokenized_context, 10)
    y = torch.argmax(y, dim=2)
    print(y)
    completion = shakespeare_dataset.to_string(y[0])
    print("completion:", completion)

KeyError: ignored

In [10]:
decoder.eval()

num_tokens_to_generate = 10

with torch.no_grad():
    tok_emb = decoder.WTE(tokenized_context)
    pos_emb = decoder.WPE(tok_emb)
    x = decoder.dropout(tok_emb + pos_emb)

    for block in decoder.Blocks:
        x = block(x)

    x = decoder.Final_LayerNorm(x)
    generated_tokens = decoder.LM_Head(x)
    generated_tokens = generated_tokens[:, -num_tokens_to_generate:]
    print(generated_tokens.shape)
decoder.train()

torch.Size([1, 10, 47])


TransformerDecoder(
  (WTE): Embedding(47, 128)
  (WPE): PositionalEncoding()
  (dropout): Dropout(p=0.1, inplace=False)
  (Blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (CausalSelfAttn): CausalSelfAttention(
        (q_linear): Linear(in_features=128, out_features=128, bias=True)
        (v_linear): Linear(in_features=128, out_features=128, bias=True)
        (k_linear): Linear(in_features=128, out_features=128, bias=True)
        (out_linear): Linear(in_features=128, out_features=128, bias=True)
      )
      (LayerNorm_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (MLP): FeedForward(
        (linear1): Linear(in_features=128, out_features=100, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=100, out_features=128, bias=True)
      )
      (LayerNorm_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
  )
  (Final_LayerNorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (LM_Head)

In [20]:
shakespeare_dataset.itos

{0: 'c',
 1: '\n',
 2: 'b',
 3: 'h',
 4: 'd',
 5: ' ',
 6: 'f',
 7: 'g',
 8: 'e',
 9: 'a'}

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
quit()

In [55]:
a = torch.tensor([[[1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6]]])
print(a.shape)
res = a.sum(dim=2)
print(res)
print(res.shape)

torch.Size([1, 6, 2])
tensor([[ 2,  4,  6,  8, 10, 12]])
torch.Size([1, 6])
